# Julia言語で入門するプログラミング

[Julia言語で入門するプログラミング](https://muuumin.net/julia-intro-prog/)

## 標準出力の自動テスト

前回までのプログラムは、標準出力メッセージ (`println()` で出力しているメッセージ) はそのまま標準出力に出力していた

しかし、本当はこれらのメッセージも自動テストでテストしておきたい

`println()` 関数は、本来は出力先の IO を第1引数に指定可能で、デフォルトでは `println(stdout, "...")` のように「標準出力に対して "..." を出力する」という形になっている

この第1引数は `IO` 型であり、`IOBuffer` オブジェクトを指定することができる

`IOBuffer` オブジェクトは、文字列やバッファを書き込んだ後 `take!()` 関数で中身のバッファを取得することができるため、以下のように書いてテストを行うことができる

```julia
"「こんにちは！」を出力する関数"
function 挨拶する(io::IO)
    println(io, "こんにちは！")
end

# 標準出力に「こんにちは！」を出力する
挨拶する(stdout) # => コンソールに「こんにちは！」が出力される

# 自動テスト
@testset "挨拶する" begin
    io = IOBuffer()
    挨拶する(io)

    # 挨拶する関数が io に書き込んだ文字列が「こんにちは！」であることを保証する
    ## println 関数は後ろに改行をつけるため「こんにちは！\n」が書き込まれているはず
    @test String(take!(io)) === "こんにちは！\n"
end
```

In [1]:
function 挨拶する(io::IO)
    println(io, "こんにちは！")
end

io = IOBuffer()
挨拶する(io)

io |> take! |> String

"こんにちは！\n"

これを使って、戦闘メッセージも自動テストできるようにプログラムを改良する

In [2]:
using Test

"再定義可能な構造体を定義するマクロ"
macro restruct(構造体宣言式)
    一時的な構造体の名前 = "var\"$(gensym())\""
    mutable = 構造体宣言式.args[1] ? "mutable" : ""
    構造体を束縛する変数名 = 構造体宣言式.args[2]
    構造体内部コード = join(構造体宣言式.args[3].args, ";")

    Meta.parse(
        "Base.@kwdef $mutable struct $一時的な構造体の名前 $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

"HP計算関数"
function HP計算(HP::Int, ダメージ量::Int)
    ダメージ後のHP = HP - ダメージ量
    if ダメージ後のHP < 0
        return 0
    end
    return ダメージ後のHP
end

"ダメージ計算関数"
function ダメージ計算(攻撃力::Int, 防御力::Int)
    if 攻撃力 < 0 || 防御力 < 0
        throw(DomainError((攻撃力, 防御力), "攻撃力, 防御力は正の値である必要があります")) 
    end
    if 防御力 === 0
        throw(DomainError(防御力, "防御力は0より大きい値である必要があります")) 
    end
    ダメージ量 = round(Int, 10 * 攻撃力 / 防御力)
end

# キャラクター可変構造体
@restruct mutable struct キャラクター
    名前::String
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

"攻撃処理関数（戦闘メッセージ自動テスト対応版）"
function 攻撃処理!(攻撃者::キャラクター, 防御者::キャラクター; io::IO = stdout)
    println(io, "----------")
    println(io, "$(攻撃者.名前)の攻撃！")
    防御者ダメージ = ダメージ計算(攻撃者.攻撃力, 防御者.防御力)
    防御者.HP = HP計算(防御者.HP, 防御者ダメージ)
    println(io, "$(防御者.名前)は $(防御者ダメージ) のダメージを受けた！")
    println(io, "$(防御者.名前)の残りHP：$(防御者.HP)")
end

"行動順決定関数"
function 行動順決定(勇者::キャラクター, モンスター::キャラクター, 乱数::Number)
    if 乱数 < 0.5
        return [(勇者, モンスター), (モンスター, 勇者)]
    else
        return [(モンスター, 勇者), (勇者, モンスター)]
    end
end

"ゲームループ処理関数（戦闘メッセージ自動テスト対応版）"
function ゲームループ!(勇者::キャラクター, モンスター::キャラクター, 乱数生成関数::Function; io::IO = stdout)
    戦闘ターン数 = 1

    while 勇者.HP > 0 && モンスター.HP > 0
        for (攻撃者, 防御者) in 行動順決定(勇者, モンスター, 乱数生成関数())
            攻撃処理!(攻撃者, 防御者; io = io)
            if 防御者.HP === 0
                return 戦闘ターン数 # ゲームループ終了
            end
        end
        戦闘ターン数 += 1
    end

    戦闘ターン数
end

"main関数（戦闘メッセージ自動テスト対応版）"
function main(乱数生成関数::Function; io::IO = stdout)
    モンスター = キャラクター("モンスター", 30, 10, 10)
    勇者 = キャラクター("勇者", 30, 10, 10)

    println(io, "モンスターに遭遇した！")
    println(io, "戦闘開始！")

    戦闘ターン数 = ゲームループ!(勇者, モンスター, 乱数生成関数; io = io)

    if モンスター.HP === 0
        println(io, "戦闘に勝利した！")
    else
        println(io, "戦闘に敗北した・・・")
    end

    戦闘ターン数, 勇者.HP, モンスター.HP
end

# test
function 疑似乱数生成高階関数(疑似乱数配列::Vector{<:Number})
    function 疑似乱数生成関数()
        popfirst!(疑似乱数配列)
    end
end

@testset "main関数" begin
    io = IOBuffer()
    @test main(疑似乱数生成高階関数([0.5, 0.4, 0.6]); io = io) === (3, 0, 10)

    # 戦闘メッセージテスト
    @test (io |> take! |> String) === """
モンスターに遭遇した！
戦闘開始！
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・
"""

    @test main(疑似乱数生成高階関数([0.4, 0.7, 0.3]); io = io) === (3, 10, 0)
    @test (io |> take! |> String) === """
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
"""
end

Test Summary: | Pass  Total
main関数      |    4      4


Test.DefaultTestSet("main関数", Any[], 4, false, false)

## 標準入力

次はプレイヤーキャラクターに命令を与えられるようにしたい

ユーザから何らかの命令を与えられるようにするには、プログラムに対してなにか入力する必要が出てくる

最も一般的な入力方法は、標準入力と呼ばれ、コンソールからキーボードで文字列を入力する方法である

標準出力はコンソールに対して文字列を出力する仕組みで、`println` 関数などで出力を行うことが出来た

一方、標準入力は `Base.prompt` 関数などで提供されており、以下のような形で利用する

```julia
Base.prompt("何か入力してください")

# => コンソールに「何か入力してください」と表示され、プログラムが停止する（入力待ち状態になる）
# => キーボードから適当な文字列を入力して Enter キーを押すとプログラムが再開する
# => 入力された文字列が戻り値として設定される
```

In [3]:
Base.prompt("何か入力してください")

"1"

### 行動の追加
今回はプレイヤーが命令を与えられるようにしたいわけだが、現状行動の選択肢が「攻撃」しかないため、選択肢を増やす

まずは「大振り」という攻撃手段を追加してみる

- 大振り:
    - 種類: 攻撃
    - 命中率: 60%
    - 攻撃倍率: 200%

In [4]:
@restruct mutable struct キャラクター
    名前::String
    isプレイヤー::Bool = false # 追加: 行動前に行動選択を促すかどうかのフラグ
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

"コマンドシンボル対応表"
行動コマンド = Dict(
    "1" => :攻撃,
    "2" => :大振り
)

"ダメージ計算（攻撃倍率、命中率対応）"
function ダメージ計算(攻撃力::Int, 防御力::Int; 攻撃倍率::AbstractFloat = 1.0, 命中率::AbstractFloat = 1.0)
    if rand() >= 命中率
        # 攻撃が外れたときは攻撃倍率を 0.0 倍に
        攻撃倍率 = 0.0
    end

    与ダメージ力::AbstractFloat = 攻撃力 * 攻撃倍率

    if 与ダメージ力 < 0 || 防御力 < 0
        throw(DomainError((攻撃力, 攻撃倍率, 防御力), "攻撃力, 攻撃倍率, 防御力は正の値である必要があります")) 
    end
    if 防御力 === 0
        throw(DomainError(防御力, "防御力は0より大きい値である必要があります")) 
    end
    ダメージ量 = round(Int, 10 * 与ダメージ力 / 防御力)
end

"攻撃処理関数（行動による戦闘メッセージ変更対応）"
function 攻撃処理!(攻撃者::キャラクター, 防御者::キャラクター, コマンド::Symbol; io::IO = stdout)
    攻撃倍率, 命中率 = 1.0, 1.0

    println(io, "----------")
    if コマンド === :攻撃
        println(io, "$(攻撃者.名前)の攻撃！")
    elseif コマンド === :大振り
        println(io, "$(攻撃者.名前)の大振り！")
        攻撃倍率, 命中率 = 2.0, 0.6
    else
        throw(DomainError(コマンド, "定義されていないコマンドが入力されました"))
    end
    防御者ダメージ = ダメージ計算(攻撃者.攻撃力, 防御者.防御力; 攻撃倍率 = 攻撃倍率, 命中率 = 命中率)
    防御者.HP = HP計算(防御者.HP, 防御者ダメージ)

    if 防御者ダメージ === 0
        println(io, "ミス！$(防御者.名前)はダメージを受けなかった")
    else
        println(io, "$(防御者.名前)は $(防御者ダメージ) のダメージを受けた！")
    end
    println(io, "$(防御者.名前)の残りHP：$(防御者.HP)")
end

"キャラクター行動処理"
function 行動!(攻撃者::キャラクター, 防御者::キャラクター; io::IO = stdout)
    if 攻撃者.isプレイヤー
        # プレイヤーキャラクターの場合は行動入力
        コマンドID = Base.prompt("[1]攻撃 [2]大振り")
        攻撃処理!(攻撃者, 防御者, 行動コマンド[コマンドID]; io)
    else
        攻撃処理!(攻撃者, 防御者, :攻撃; io)
    end
end

"行動順決定関数"
function 行動順決定(勇者::キャラクター, モンスター::キャラクター, 乱数::Number)
    if 乱数 < 0.5
        return [(勇者, モンスター), (モンスター, 勇者)]
    else
        return [(モンスター, 勇者), (勇者, モンスター)]
    end
end

"ゲームループ処理関数"
function ゲームループ!(勇者::キャラクター, モンスター::キャラクター; io::IO = stdout)
    戦闘ターン数 = 1

    while 勇者.HP > 0 && モンスター.HP > 0
        for (攻撃者, 防御者) in 行動順決定(勇者, モンスター, rand())
            行動!(攻撃者, 防御者; io = io)
            if 防御者.HP === 0
                return 戦闘ターン数 # ゲームループ終了
            end
        end
        戦闘ターン数 += 1
    end

    戦闘ターン数
end

# 動作確認
(() -> begin
    勇者 = キャラクター("勇者", true, 30, 10, 10)
    モンスター = キャラクター("モンスター", false, 30, 10, 10)

    ゲームループ!(勇者, モンスター)
end)()

----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の大振り！
モンスターは 20 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の大振り！
モンスターは 20 のダメージを受けた！
モンスターの残りHP：0


2

行動選択の入力ができることを確認できたら完了である

## テスト駆動開発

テスト駆動開発は一風変わった開発方法である

普通はコードを書いてからテストをするが、このやり方はテストを書いてからコードを作る

妙なやり方に思えるかもしれないが、慣れるとむしろコードから先に書くやり方だと落ち着かなくなってくる

テスト駆動開発のサイクルは以下の通りである

1. まず最初にテストコードを書く
2. 何もないところにテストコードだけ書くので必ず失敗する（失敗することを確認する）
3. テストが通る最低限のコードを書く
4. テストが通ることを確認する
5. 1 に戻って新たなテストを書く

思いつく限りのテストを書いたら完了である

このようなやり方をする理由はいくつかあり、主に以下のようなものがある

- 関数のインターフェースが自然になる
    - 関数を作る時に実装から入ってしまうと、既存の部品に合わせたインターフェース(引数や返り値)になってしまうことがある
    - それがその関数にとって自然なインターフェースであれば良いのだが、そうならないこともある
    - テストから書く＝関数の呼び出し方を決める、ということなので、その関数にとって最も自然なインターフェースをまず考えることになる
- 不必要に複雑な実装にならない
    - テストが通る最低限のコードを書くというところが肝要である
    - コードを書いていると、こんな拡張性はいるかな、あんなことは考慮しておいたほうがいいかな、と考えすぎて無駄に複雑な設計になってしまうことがある
    - テストが通る最低限のコードを書くことは、設計が複雑になりすぎない重要な基準になる
    - もちろん、テストが足りなければ単に欠陥のあるコードなので、テストは十分に用意しておく必要がある
- テストが後回しにされない
    - コードは実装があれば動く。テストがなくても動く。だから、気を抜くとテストは作られなくなってしまう
    - テスト駆動開発ではテストから始めるため、コードの安全性を担保しやすい
- テスト可能な関数になる
    - 自動テスト可能なコードというのは自然には出来上がらない
    - 途中からテスト可能なコードに作り替えるのは大変であるため、最初から作っておけば無駄な手戻りを防ぐことができる
- テストケースが関数の仕様となる
    - テストコードを見ることで、どのような使い方が想定された関数かがわかるようになる
    - 動作保証されているサンプルコードとなるわけである

このようにメリットの多い手法なので、ぜひ身につけていきたい

### 準備
テストコードを作り始める前に、ソースコードの管理の仕方をパッケージベースに変更する

Julia では、プロジェクト単位のソースコードとテストコードをまとめたものを **パッケージ** と呼ぶ

パッケージのテンプレートは `Pkg.generate()` 関数で作成でき、以下のようなディレクトリ構成となっている

```bash
{ProjectName}
|_ src/ # ソースコード管理ディレクトリ
|  |_ {ProjectName}.jl # メインソースコード
|
|_ test/ # 自動テストコード管理ディレクトリ
|  |_ runtests.jl # Pkg.test 関数で実行されるメイン自動テストコード
|
|_ Project.toml  # プロジェクト名や依存パッケージ等を記述する設定ファイル
|_ Manifest.toml # 依存パッケージのバージョン情報等を記述する設定ファイル
```

なお、`test` ディレクトリは `Pkg.generate()` 関数では作られないため、自分で作成する必要がある（`Manifest.toml` は依存パッケージインストール時に自動で作成される）

今回は、[04](./04) ディレクトリに、プロジェクト名（パッケージ名）: `RPG` としてパッケージを作成する

In [5]:
using Pkg

# カレントディレクトリに 04 ディレクトリがなければ
## 04 ディレクトリ作成して RPG パッケージテンプレート配置
if !isdir("./04")
    Pkg.generate("./04/RPG")
end

### 型定義
プログラムは基本的に「データの操作を行う」ものである

すなわち、数値や文字列といった「データ」に対して、計算や入力・出力といった「操作」を行うものがプログラムである

そのためプログラムの設計を行う際は、「どのようなデータ」に対して「どのような操作」を行うのかを考えていくと比較的やりやすい

まずは、「どのようなデータ」が必要になるのか、その整理を行い構造体にまとめていく（型定義を行う）

現状 RPG ゲームに必要なデータを整理すると以下のようになる

- [ゲーム](./04/RPG/src/struct/Game.jl)
    - `勇者::キャラクター`
    - `モンスター::キャラクター`
- [キャラクター](./04/RPG/src/struct/Character.jl)
    - `名前::String`
    - `HP::Int`
    - `攻撃力::Int`
    - `防御力::Int`
- [コマンド](./04/RPG/src/Command.jl)
    - 攻撃 (1)
    - 大振り (2)

### テストケースの整理
次に、整理した各データに対して「どのような操作」を行うのかを考えていく

このとき、処理のテストケースも合わせて考えていく（テストケース＝仕様設計 である）

また、1つの処理が長すぎたり、条件分岐が何度も発生する場合には、適切に処理を分解することが肝要である（処理を分けるポイント＝テストケースの書きやすさ である）

#### ゲームに対する処理
- `戦闘開始処理`:
    1. 「モンスターに遭遇した！`\n`戦闘開始！`\n`」のメッセージが出力されること
- `戦闘処理（1ターンの処理）`: ※ この処理は他の処理を呼び出しているだけなので今回はテストしない
    1. `先行後攻決定処理` で先攻・後攻: `ゲーム.勇者` or `ゲーム.モンスター` が決定されること
    2. 1 で取得した配列 `[1]` に対して `行動処理(攻撃者, 防御者)` が行われること
    3. `戦闘終了判断` の戻り値に対する条件分岐:
        - ◇ true の場合:
            - 処理が終了されること
        - ◇ false の場合:
            - 処理が継続されること
    4. 1 で取得した配列 `[2]` に対して `行動処理(攻撃者, 防御者)` が行われること
- `戦闘処理（ループ処理）`: ※ この処理は他の処理を呼び出しているだけなので今回はテストしない
    1. `戦闘終了判断` の戻り値に対する条件分岐:
        - ◇ true の場合:
            - 処理が終了されること
        - ◇ false の場合:
            - 処理が継続されること
    2. `戦闘処理（1ターンの処理）` が実行されること
    3. 1 に戻ること
- `戦闘終了処理`:
    1. 条件分岐:
        - ◇ `ゲーム.モンスター.HP` が 0 の場合:
            - 「戦闘に勝利した！`\n`」のメッセージが出力されること
        - ◇ `ゲーム.勇者.HP` が 0 の場合:
            - 「戦闘に敗北した・・・`\n`」のメッセージが出力されること
        - ◇ 上記以外の場合:
            - 例外処理が行われること (モンスターか勇者のHPが 0 にならないと呼ばれないはずのため)
- `先行後攻決定処理`:
    1. 確率分岐:
        - ◇ 50% の確率:
            - `[(ゲーム.勇者, ゲーム.モンスター), (ゲーム.モンスター, ゲーム.勇者)]` の順の配列が返ること
        - ◇ 50% の確率:
            - `[(ゲーム.モンスター, ゲーム.勇者), (ゲーム.勇者, ゲーム.モンスター)]` の順の配列が返ること

#### キャラクターに対する処理
- `行動処理(攻撃者, 防御者)`: ※ この処理は他の処理を呼び出しているだけなので今回はテストしない
    1. 条件分岐:
        - ◇ `攻撃者.isプレイヤー` が true の場合:
            - `行動入力処理` が実行されること
        - ◇ `攻撃者.isプレイヤー` が false の場合:
            - `行動選択処理` が実行されること
    2. 1 で戻り値に設定されたコマンドに対する条件分岐:
        - ◇ コマンド `攻撃` の場合:
            - `攻撃処理(攻撃者, 防御者)` が実行されること
        - ◇ コマンド `大振り` の場合:
            - `大振り処理(攻撃者, 防御者)` が実行されること
        - ◇ 上記以外の場合:
            - 例外処理が行われること
- `行動入力処理`: ※ プレイヤーキャラクターの行動選択
    1. 標準入力処理が行われること
    2. 1 で入力された文字列に対する条件分岐:
        - ◇ 「1」が入力された場合:
            - コマンド `攻撃` が戻り値に設定されること
        - ◇ 「2」が入力された場合:
            - コマンド `大振り` が戻り値に設定されること
        - ◇ 上記以外の場合:
            - 例外処理が行われること
- `行動選択処理`: ※ モンスターの行動選択
    1. コマンド `攻撃` が戻り値に設定されること

#### キャラクターに対する処理（コマンド処理）
- `攻撃処理(攻撃者, 防御者)`:  ※ この処理は他の処理を呼び出しているだけなので今回はテストしない
    1. `行動メッセージ出力(攻撃者.名前, "攻撃")` が実行されること
    2. `攻撃ダメージ計算処理(攻撃者.攻撃力, 防御者.防御力)` が実行されること
    3. `ダメージ処理(防御者, 2 の戻り値)` が実行されること
    4. `ダメージメッセージ出力(防御者, 2 の戻り値)` が実行されること
- `大振り処理(攻撃者, 防御者)`:
    1. `行動メッセージ出力(攻撃者.名前, "大振り攻撃")` が実行されること
    2. 確率分岐:
        - ◇ 60% の確率:
            - `攻撃ダメージ計算処理(攻撃者.攻撃力 * 2, 防御者.防御力)` が実行されること
        - ◇ 40% の確率:
            - `攻撃ダメージ計算処理(0, 防御者.防御力)` が実行されること
    3. `ダメージ処理(防御者, 2 の戻り値)` が実行されること
    4. `ダメージメッセージ出力(防御者, 2 の戻り値)` が実行されること
- `行動メッセージ出力(名前, 行動名)`:
    1. 「----------`\n`{名前}の{行動名}！`\n`」のメッセージが出力されること
- `ダメージメッセージ出力(防御者, ダメージ量)`:
    1. 条件分岐:
        - ◇ ダメージ量が 0 の場合:
            - 「ミス！{防御者.名前}はダメージを受けなかった`\n`」のメッセージが出力されること
        - ◇ 上記以外の場合:
            - 「{防御者.名前}は {ダメージ量} のダメージを受けた！`\n`」のメッセージが出力されること
    2. 「{防御者.名前}のHP: {防御者.HP}`\n`」のメッセージが出力されること
- `攻撃ダメージ計算処理(攻撃力, 防御力)`:
    1. 計算処理: $10 \times \frac{攻撃力}{防御力}$ (小数点以下四捨五入)
        - テストケース: (攻撃力, 防御力, 計算結果)
            - ◇ (10, 10, 10)
            - ◇ (14, 100, 1)
            - ◇ (15, 100, 2)
            - ◇ (0, 10, 0)
            - ◇ (-10, 10, 例外)
            - ◇ (10, -10, 例外)
            - ◇ (-10, -10, 例外)
            - ◇ (10, 0, 例外)
- `ダメージ処理(防御者, ダメージ量)`:
    1. 防御者.HP がダメージ量分減算されること
        - ◇ ダメージ量がマイナスのとき:
            - 例外処理が行われること
    2. 条件分岐:
        - ◇ 防御者.HP が 0 未満の場合:
            - 防御者.HP が 0 に設定されること
        - ◇ 上記以外の場合:
            - N/A

### テストの実行
[テストコード](./04/RPG/test/runtests.jl) を記述し、テストコードを通すための最低限のプログラムコードを実装したら、自動テストを実行する

これまでは、グローバルプロジェクトでテストを実行していたため、Test パッケージはデフォルトで入っていた

しかし、今回のように新規にプロジェクトを作成した場合、そのプロジェクト用に必要なパッケージをインストールする必要があるため注意する

テスト実行までの手順をまとめると以下のようになる

1. 環境切り替え
    - `Pkg.activate()` 関数でグローバルプロジェクトから対象プロジェクトに環境を切り替える
2. 必要なパッケージのインストール
    - `Pkg.add()` 関数で必要なパッケージをインストールする
    - Test パッケージなどの標準パッケージもインストールしなければならないため注意
3. 自動テストの実行
    - `Pkg.test()` 関数で `test/runtests.jl` に記述されたテストコードが実行される

In [6]:
# プロジェクト環境を ./04/RPG/ に切り替え
Pkg.activate("./04/RPG")

# Test パッケージのインストール
## => Project.toml, Manifest.toml に必要な情報が記述される
Pkg.add("Test")

  Activating project at `d:\github\julia_ml-tuto\01_tutorial\RPG\04\RPG`
    Updating registry at `C:\Users\user\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `D:\github\julia_ml-tuto\01_tutorial\RPG\04\RPG\Project.toml`
  No Changes to `D:\github\julia_ml-tuto\01_tutorial\RPG\04\RPG\Manifest.toml`


In [7]:
# 自動テストの実行
Pkg.test()

Test Summary: | Pass  Total
戦闘開始処理  |    1      1
Test Summary: | Pass  Total
戦闘終了処理  |    3      3
Test Summary: | Pass  Total
先行後攻決定処理 |    2      2
Test Summary: | Pass  Total
行動入力処理  |    3      3
Test Summary: | Pass  Total
行動選択処理  |    1      1
Test Summary: | Pass  Total
行動メッセージ処理 |    1      1
Test Summary: | Pass  Total
ダメージメッセージ処理 |    2      2
Test Summary: | Pass  Total
攻撃ダメージ計算処理 |    8      8
Test Summary: | Pass  Total
ダメージ処理! |    5      5
Test Summary: | Pass  Total
大振り攻撃処理! |    4      4


     Testing RPG
      Status `C:\Users\user\AppData\Local\Temp\jl_E9OTKe\Project.toml`
  [b4d49c43] RPG v0.1.0 `D:\github\julia_ml-tuto\01_tutorial\RPG\04\RPG`
  [8dfed614] Test `@stdlib/Test`
      Status `C:\Users\user\AppData\Local\Temp\jl_E9OTKe\Manifest.toml`
  [b4d49c43] RPG v0.1.0 `D:\github\julia_ml-tuto\01_tutorial\RPG\04\RPG`
  [2a0f44e3] Base64 `@stdlib/Base64`
  [b77e0a4c] InteractiveUtils `@stdlib/InteractiveUtils`
  [56ddb016] Logging `@stdlib/Logging`
  [d6f4376e] Markdown `@stdlib/Markdown`
  [9a3f8284] Random `@stdlib/Random`
  [ea8e919c] SHA `@stdlib/SHA`
  [9e88b42a] Serialization `@stdlib/Serialization`
  [8dfed614] Test `@stdlib/Test`
     Testing Running tests...
     Testing RPG tests passed 
